## Load packages and data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
test = pd.read_csv('../input/porto-seguro-safe-driver-prediction/test.csv')

print(train.shape)
print(test.shape)

## Preprocessing

 + Missing value가 많은 변수는 제거한다.

In [3]:
# check missing percentage
train = train.replace(-1, np.nan)
na_count = train.isna().sum().sort_values(ascending = False).reset_index()
na_count.rename(columns = {'index':'variable', 0: 'count'}, inplace = True)
na_count['prop'] = na_count['count']/train.shape[0]
na_count[na_count['count']>0]

In [4]:
cols_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train.drop(cols_to_drop, axis = 1, inplace = True)
test.drop(cols_to_drop, axis = 1, inplace = True)

# missing value가 많은 변수 제거
vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']
train.drop(vars_to_drop, inplace=True, axis=1)
test.drop(vars_to_drop, inplace=True, axis=1)

In [5]:
# 이전 커널과 똑같은 target encoding 기법
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

 + 카테고리가 지나치게 많은 변수(high cardinality)를 Target encoding 한다.

In [6]:
# high cardinality 특성을 가진 변수 Target Encoding
print('The number of categories: ', train['ps_car_11_cat'].nunique())

train_encoded, test_encoded = target_encode(train["ps_car_11_cat"], 
                             test["ps_car_11_cat"], 
                             target=train.target, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0.01)
    
train['ps_car_11_cat_te'] = train_encoded
train.drop('ps_car_11_cat', axis=1, inplace=True)
test['ps_car_11_cat_te'] = test_encoded
test.drop('ps_car_11_cat', axis=1, inplace=True)

## Undersampling
 + Target의 클래스 1과 0의 비율 차이가 매우 크므로(imbalanced), undersampling을 진행한다.
  + Undersampling을 할 경우, overfitting을 방지할 순 있지만 데이터의 손실이 발생해 정확도가 떨어질 수 있다는 문제점이 있다.

In [7]:
desired_apriori=0.10

idx_0 = train[train['target'] == 0].index
idx_1 = train[train['target'] == 1].index

nb_0 = len(train.loc[idx_0])
nb_1 = len(train.loc[idx_1])

undersampling_rate = ((1-desired_apriori)*nb_1)/(nb_0*desired_apriori)
undersampled_nb_0 = int(undersampling_rate*nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

under_idx = shuffle(idx_0, n_samples = undersampled_nb_0, random_state = 0)
idx_list  = list(under_idx) + list(idx_1)

train = train.loc[idx_list].reset_index(drop = True)

In [8]:
targets = pd.DataFrame(train['target'].value_counts().reset_index())
targets['prop'] = targets['target']/train.shape[0]

plt.figure(figsize = (6, 4))
ax = plt.subplot(1, 1, 1)
sns.barplot(targets['index'], targets['target'], palette = ['#006699', '#cc0000'], alpha = 0.6, edgecolor = 'k')
for s in ["top","right","left"]:
    ax.spines[s].set_visible(False)
ax.grid(axis='y', linestyle='-', alpha=0.4)

plt.title('Target Distribution after Undersampling', fontweight = 'bold', fontsize = 15, pad = 20)
plt.xlabel('Target', fontweight = 'bold'); plt.ylabel('Count', fontweight = 'bold')
plt.show()

In [9]:
# replace -1 with NaN
train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)

# One-hot encoding: categorical variables
cat_features = [a for a in train.columns if a.endswith('cat')]

for column in cat_features:
    temp = pd.get_dummies(pd.Series(train[column]))
    train = pd.concat([train,temp],axis=1)
    train = train.drop([column],axis=1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    test = pd.concat([test,temp],axis=1)
    test = test.drop([column],axis=1)
    
id_test = test['id'].values
target_train = train['target'].values

train = train.drop(['target','id'], axis = 1)
test = test.drop(['id'], axis = 1)

In [10]:
# check final data
print('Train set: {}'.format(train.shape))
print('Test set: {}'.format(test.shape))

## Stacking Ensemble
1. 각 모델별로 원본 train/test set을 예측한 결과값을 기반으로 메타 모델을 위한 meta train/test set을 생성한다.(자세한 내용은 스터디 때 공유)
2. 1에서 개별 모델들이 생성한 train set을 모두 stacking 형태로 합쳐서 메타 모델이 학습할 최종 메타 데이터셋을 생성한다. 

 + 기본적인 Stacking을 사용할 경우, overfitting 문제가 발생할 수 있기 때문에 CV 기반 Stacking을 주로 많이 사용한다.

#### 간단한 Gini 계수 계산법 - Gini 계수와 ROC/AUC의 연관
<https://luckytoilet.wordpress.com/2018/04/04/useful-properties-of-roc-curves-auc-scoring-and-gini-coefficients/>

In [ ]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models
        
    def fit_predict(self, X, y, T):
        X = np.array(X) # X_train
        y = np.array(y) # y_train
        T = np.array(T) # test set
        
        
        folds = list(StratifiedKFold(n_splits = self.n_splits, shuffle = True, random_state = 0).split(X, y))
        
        # meta train, test for stacking model
        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            # 각 폴드별 meta test set 생성
            S_test_i = np.zeros((T.shape[0], self.n_splits))
            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
                
                print("Base model %d: fit %s model | fold %d" %(i+1, str(model.__class__.__name__), j+1))
                
                model.fit(X_train, y_train)
                cross_score = cross_val_score(model, X_train, y_train, cv = 3, scoring = 'roc_auc')
                print("cross_score [roc-auc]: %.5f [gini]: %.5f" % (cross_score.mean(), 2*cross_score.mean()-1))
                
                y_pred = model.predict_proba(X_holdout)[:,1]
                S_train[test_idx, i] = y_pred # meta train set
                S_test_i[:, j] = model.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis = 1) # meta test set: 각 fold에서 구한 probability의 평균 
            
        results = cross_val_score(self.stacker, S_train, y, cv = 3, scoring = 'roc_auc')
        print("Stacker score [gini]: %.5f" % (2 * results.mean() - 1))
        
        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:, 1]
        return res

In [21]:
# LightGBM params
# lgb_1
lgb_params1 = {}
lgb_params1['learning_rate'] = 0.02
lgb_params1['n_estimators'] = 650
lgb_params1['max_bin'] = 10
lgb_params1['subsample'] = 0.8
lgb_params1['subsample_freq'] = 10
lgb_params1['colsample_bytree'] = 0.8   
lgb_params1['min_child_samples'] = 500
lgb_params1['seed'] = 314
lgb_params1['num_threads'] = 4

# lgb2
lgb_params2 = {}
lgb_params2['n_estimators'] = 1090
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3   
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 314
lgb_params2['num_threads'] = 4

# lgb3
lgb_params3 = {}
lgb_params3['n_estimators'] = 1100
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 314
lgb_params3['num_threads'] = 4

# XGBoost params
xgb_params = {}
xgb_params['objective'] = 'binary:logistic'
xgb_params['learning_rate'] = 0.04
xgb_params['n_estimators'] = 490
xgb_params['max_depth'] = 4
xgb_params['subsample'] = 0.9
xgb_params['colsample_bytree'] = 0.9  
xgb_params['min_child_weight'] = 10
xgb_params['num_threads'] = 4

In [17]:
# Base models
lgb_model1 = LGBMClassifier(**lgb_params1)
lgb_model2 = LGBMClassifier(**lgb_params2)
lgb_model3 = LGBMClassifier(**lgb_params3)
xgb_model = XGBClassifier(**xgb_params)

# Stacking model - response가 binary이므로 LR 모델 생성
log_model = LogisticRegression()

In [19]:
stack = Ensemble(n_splits = 3,
        stacker = log_model,
        base_models = (lgb_model1, lgb_model2, lgb_model3, xgb_model))  

In [20]:
y_prediction = stack.fit_predict(train, target_train, test) 